In [ ]:
class Node:
    def __init__(parent):
        self.parent = parent
        self.left = None
        self.right = None
        self.balance_factor = 0
        self.height = 0
    
    def set_right(right):
        if self.right is not None:
            old_height = self.right.height
        else:
            old_height = -1
        self.right = right
        self.balance_factor += self.right.height - old_height
        
    def set_left(left):
        self.left = left

In [ ]:
def calc_height